## Seguimiento utilizando Camshift

In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: <object object at 0x000002063B21CB30>


In [2]:
def mousePoints(event,x,y,flags,params):

    # Left button mouse click event opencv
    if (event == cv.EVENT_LBUTTONDOWN) and (len(roi_points)<2):
        roi_points.append((x,y))
        

In [3]:
def ROI_init(roi_p):

    w = np.abs(roi_p[0][0]-roi_p[1][0])
    h = np.abs(roi_p[0][1]-roi_p[1][1])
    x = np.min([roi_p[0][0],roi_p[1][0]])
    y = np.min([roi_p[0][1],roi_p[1][1]])
    
    # Definimos la ubicación inical de la ventana de seguimiento
    #-----------------------------------------------------------
    # r,h,c,w - Región de la imagen (valores harcodeados)
    track_window = (x, y, w, h)

    # Una vez definida la ROI recortamos esa parte de la imagen que utilizaremos para seguimiento
    #--------------------------------------------------------------------------------------------
    roi = frame[y:y+h, x:x+w]
    # Pasamos la ROI a HSV (los valores de H son más estables a cambios de intensidad)
    hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
    # Definimos una máscara para el histograma (Hue: 0..180, Saturation:60..255, Value:32..255)
    mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
    # Y calculamos el histograma sobre esa máscara (toma solo el Hue: tonalidad)
    roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
    # Normalizamos el histograma (entre 0 y 255 según indica el flag cv.NORM_MINMAX)
    cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

    cv.imwrite("ROI_2.jpg",mask)

    # Mostramos lo que se fue armando en relación a la ROI
    #-----------------------------------------------------
    plt.figure()
    plt.imshow(mask)

    # Muestro la imagen como para ver dónde definimos la ROI
    #-------------------------------------------------------
    cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2,cv.LINE_AA)
    plt.figure()
    plt.imshow(cv.cvtColor(frame,cv.COLOR_BGR2RGB))
    cv.imwrite("frame_ROI_2.jpg",frame)
    
    return track_window, roi_hist

In [4]:
cap = cv.VideoCapture(0)
fourcc = cv.VideoWriter_fourcc(*'MP4V')
video = cv.VideoWriter('Camshift_2.mp4', fourcc, 20.0, (640,480))


if(cap.isOpened()==False):
    print("Error opening camera")
    
event = 0
flag_set_roi = True
roi_points = []

# Condición de terminación: 10 iteraciones o moverse al menos 1pt
#----------------------------------------------------------------
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10 )

while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret == True:
        cv.setMouseCallback("Frame", mousePoints)
        
        if (len(roi_points)==2) and (flag_set_roi==True):
            print(roi_points)
            track_window, roi_hist = ROI_init(roi_points)
            flag_set_roi = False
        
        if flag_set_roi==False:
            hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
            dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)

            # Aplicamos camshift al nuevo frame
            #----------------------------------
            ret, track_window = cv.CamShift(dst, track_window, term_crit)

            # Dibujamos la ventana sobre la imagen
            #=====================================

            # Con escalamiento y rotación de la ventana de seguimiento
            #---------------------------------------------------------
            pts = cv.boxPoints(ret)                      # Encuentra los 4 vértices del rectángulo rotado
            pts = np.int0(pts)                           # int0 (entero usado para indexado)
            img2 = cv.polylines(frame,[pts],True, 255,2) # Dibujo del rectángulo

            cv.imshow('Frame',img2)
        else:
            cv.imshow('Frame',frame)        
        
        video.write(frame)
        k = cv.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cap.release()
video.release()
cv.destroyAllWindows()
roi_points = []

Error opening camera
